In [1]:
#had a lot of issues trying to install the "imputeLCMD" package, so this was how I did it
# if (!requireNamespace("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install(c("impute", "pcaMethods"))

# install.packages("imputeLCMD")

In [2]:
setwd("/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.3. Analyses/P1003.3.1. Data Imputation/Input")
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1003. Ozone Sterols Healthy Asthma/P1003.3. Analyses/P1003.3.1. Data Imputation/Output')
cur_date = "022122"

library(missForest)
library(imputeLCMD)
library(readxl)
library(tidyverse)

#reading in files
ozone_data = data.frame(read_excel("Ozone_Sterol_Asthma_data_011822.xlsx", sheet = 2)) 

Loading required package: randomForest

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.

Loading required package: foreach

Loading required package: itertools

Loading required package: iterators

Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix

Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

Loading required package: pcaMethods

Loading required package: Biobase

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following object is masked from ‘package:Matrix’:

    which


The following object is masked from ‘package:rand

Goal is to generate (impute) values for missing data. Given that this dataset contains various categories (ie. cell differentials, cytokines) a different method will be employed for each.

> Lung Function Data: Random Forest (RF)

> Soluble Mediators (everything else): Quantile Regression Imputation of Left-Censored Data (QRILC)

**Also make note of subjects/ variables with missing data.

In [3]:
head(ozone_data)

,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,4,1,AS_1,AS,P,Post,NR,NR,Lung_Function,FEV1,4.3899999999999899
2,4,1,AS_1,AS,P,Pre,NR,NR,Lung_Function,FEV1,4.5999999999999899
3,4,1,AS_1,AS,S,Post,NR,NR,Lung_Function,FEV1,4.3899999999999899
4,4,1,AS_1,AS,S,Pre,NR,NR,Lung_Function,FEV1,NA
5,7,2,AS_2,AS,P,Post,R,NR,Lung_Function,FEV1,2.96
6,7,2,AS_2,AS,P,Pre,R,NR,Lung_Function,FEV1,3.08


In [4]:
#initally counting the amount of data that needs to be imputed
amount_of_imputed_data = ozone_data %>%
    mutate(count = ifelse(Value != "NA", 1, 0)) %>%
    group_by(Sample_Type, Time_Point, Variable) %>%
    summarize(Subject_Presence_Percentage = sum(count)/ 24 * 100) %>%
    arrange(Subject_Presence_Percentage)
head(amount_of_imputed_data)

`summarise()` has grouped output by 'Sample_Type', 'Time_Point'. You can override using the `.groups` argument.



Sample_Type,Time_Point,Variable,Subject_Presence_Percentage
<chr>,<chr>,<chr>,<dbl>
P,Post,Utero,0.00000
P,Pre,Utero,0.00000
S,Post,MPO,87.50000
S,Pre,MPO,87.50000
S,Post,FEV1_Change,91.66667
S,Post,FVC1_Change,91.66667


In [5]:
#implementing background filter where a variable needs to be present in at least 25% of subjects
variables_to_remove_df = ozone_data %>%
    filter(Sample_Type == "P" & Variable == "Utero")

`%notin%` <- Negate(`%in%`)
ozone_data = ozone_data[ozone_data %notin% variables_to_remove_df[1:11],]

In [6]:
ozone_data = ozone_data %>%
    #making value col numeric instead of character
    mutate(Value = ifelse(Value == "NA", NA, as.numeric(Value))) 

head(ozone_data)

Warning message in ifelse(Value == "NA", NA, as.numeric(Value)):
“NAs introduced by coercion”


,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,4,1,AS_1,AS,P,Post,NR,NR,Lung_Function,FEV1,4.39
2,4,1,AS_1,AS,P,Pre,NR,NR,Lung_Function,FEV1,4.60
3,4,1,AS_1,AS,S,Post,NR,NR,Lung_Function,FEV1,4.39
4,4,1,AS_1,AS,S,Pre,NR,NR,Lung_Function,FEV1,NA
5,7,2,AS_2,AS,P,Post,R,NR,Lung_Function,FEV1,2.96
6,7,2,AS_2,AS,P,Pre,R,NR,Lung_Function,FEV1,3.08


In [7]:
#now splitting dataset based on category, sample, time point
preimputed_df = ozone_data %>%
    group_by(Category, Sample_Type, Time_Point) %>%
    group_split()

cell_differential_plasma_post_df = preimputed_df[[1]]
cell_differential_plasma_pre_df = preimputed_df[[2]]
cell_differential_sputum_post_df = preimputed_df[[3]]
cell_differential_sputum_pre_df = preimputed_df[[4]]
cytokine_plasma_post_df = preimputed_df[[5]]
cytokine_plasma_pre_df = preimputed_df[[6]]
cytokine_sputum_post_df = preimputed_df[[7]]
cytokine_sputum_pre_df = preimputed_df[[8]]
lung_function_plasma_post_df = preimputed_df[[9]]
lung_function_plasma_pre_df = preimputed_df[[10]]
lung_function_sputum_post_df = preimputed_df[[11]]
lung_function_sputum_pre_df = preimputed_df[[12]]
sterol_plasma_post_df = preimputed_df[[13]]
sterol_plasma_pre_df = preimputed_df[[14]]
sterol_sputum_post_df = preimputed_df[[15]]
sterol_sputum_pre_df = preimputed_df[[16]]

# Random Forest Imputation

In [8]:
head(lung_function_plasma_post_df)

Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
4,1,AS_1,AS,P,Post,NR,NR,Lung_Function,FEV1,4.39
7,2,AS_2,AS,P,Post,R,NR,Lung_Function,FEV1,2.96
11,3,AS_3,AS,P,Post,NR,R,Lung_Function,FEV1,4.24
13,4,AS_4,AS,P,Post,R,NR,Lung_Function,FEV1,4.99
15,5,AS_5,AS,P,Post,R,NR,Lung_Function,FEV1,2.93
16,6,AS_6,AS,P,Post,NR,NR,Lung_Function,FEV1,2.83


In [9]:
#imputing lung function data using RF
random_forest_imputation = function(dataset){
    wider_dataset = dataset %>%
        pivot_wider(names_from = Variable, values_from = Value)
    imputed_RF_object = missForest(as.matrix(wider_dataset[,10:14]))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_final_df = data.frame(cbind(wider_dataset[,1:9], imputed_RF_df)) #%>%
        #pivot_longer(cols = 10:14, names_to = "Variable", values_to = "Value")
    
    return(imputed_final_df)
}

#calling fn
imputed_lung_function_plasma_post_df = random_forest_imputation(lung_function_plasma_post_df)
imputed_lung_function_plasma_pre_df = random_forest_imputation(lung_function_plasma_pre_df)
imputed_lung_function_sputum_post_df = random_forest_imputation(lung_function_sputum_post_df)
imputed_lung_function_sputum_pre_df = random_forest_imputation(lung_function_sputum_pre_df)

head(imputed_lung_function_plasma_post_df)

  missForest iteration 1 in progress...done!
  missForest iteration 2 in progress...done!
  missForest iteration 1 in progress...done!
  missForest iteration 2 in progress...done!
  missForest iteration 1 in progress...done!
  missForest iteration 2 in progress...done!
  missForest iteration 3 in progress...done!
  missForest iteration 4 in progress...done!
  missForest iteration 1 in progress...done!
  missForest iteration 2 in progress...done!
  missForest iteration 3 in progress...done!
  missForest iteration 4 in progress...done!


,Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,FEV1,FEV1_Percent,FEV1_Change,FVC1,FVC1_Percent
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,4,1,AS_1,AS,P,Post,NR,NR,Lung_Function,4.39,82,-4,4.41,68
2,7,2,AS_2,AS,P,Post,R,NR,Lung_Function,2.96,99,-4,3.17,88
3,11,3,AS_3,AS,P,Post,NR,R,Lung_Function,4.24,87,-5,4.37,77
4,13,4,AS_4,AS,P,Post,R,NR,Lung_Function,4.99,110,2,5.26,100
5,15,5,AS_5,AS,P,Post,R,NR,Lung_Function,2.93,83,1,3.07,78
6,16,6,AS_6,AS,P,Post,NR,NR,Lung_Function,2.83,99,0,2.86,87


# QRILC Imputation

In [10]:
head(cell_differential_plasma_post_df)

Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Sample_Weight,817
7,2,AS_2,AS,P,Post,R,NR,Cell_Differential,Sample_Weight,7992
11,3,AS_3,AS,P,Post,NR,R,Cell_Differential,Sample_Weight,2791
13,4,AS_4,AS,P,Post,R,NR,Cell_Differential,Sample_Weight,3530
15,5,AS_5,AS,P,Post,R,NR,Cell_Differential,Sample_Weight,3830
16,6,AS_6,AS,P,Post,NR,NR,Cell_Differential,Sample_Weight,3287


In [33]:
#imputing lung function data using RF
QRILC_imputation = function(dataset, number_of_variables){
    wider_dataset = dataset %>%
        pivot_wider(names_from = Variable, values_from = Value)
    
    #normalizing data since that what the QRILC function wants
    #had to pseudo log transform to prevent Inf values
    QRILC_prep = wider_dataset[,10:all_of(number_of_variables)] %>%
         mutate_all(., function(x) log10(x + 1)) %>%
         as.matrix()
    imputed_QRILC_object = impute.QRILC(QRILC_prep, tune.sigma = 0.001)
    QRILC_log10_df = data.frame(imputed_QRILC_object[1]) 
    
    #converting back the original scale
    QRILC_df = QRILC_log10_df %>%
        mutate_all(., function(x) 10^x - 1)
                   
    imputed_dataset = data.frame(cbind(dataset[,1:9], QRILC_df)) %>%
        pivot_longer(cols = 10:all_of(number_of_variables), names_to = "Variable", values_to = "Value")
    
    return(imputed_dataset)
}

#calling fn
imputed_cell_differential_plasma_post_df = QRILC_imputation(cell_differential_plasma_post_df, 24)
imputed_cell_differential_plasma_pre_df = QRILC_imputation(cell_differential_plasma_pre_df, 24)
imputed_cell_differential_sputum_post_df = QRILC_imputation(cell_differential_sputum_post_df, 24)
imputed_cell_differential_sputum_pre_df = QRILC_imputation(cell_differential_sputum_pre_df, 24)
imputed_cytokine_plasma_post_df = QRILC_imputation(cytokine_plasma_post_df, 39)
imputed_cytokine_plasma_pre_df = QRILC_imputation(cytokine_plasma_pre_df, 39)
imputed_cytokine_sputum_post_df = QRILC_imputation(cytokine_sputum_post_df, 39)
imputed_cytokine_sputum_pre_df = QRILC_imputation(cytokine_sputum_pre_df, 39)
imputed_sterol_plasma_post_df = QRILC_imputation(sterol_plasma_post_df, 27)
imputed_sterol_plasma_pre_df = QRILC_imputation(sterol_plasma_pre_df, 27)
imputed_sterol_sputum_post_df = QRILC_imputation(sterol_sputum_post_df, 27)
imputed_sterol_sputum_pre_df = QRILC_imputation(sterol_sputum_pre_df, 27)

Warning message in (function (x) :
“NaNs produced”
Warning message in (function (x) :
“NaNs produced”
Warning message in (function (x) :
“NaNs produced”
Warning message in (function (x) :
“NaNs produced”


ERROR: Error in checkSymmetricPositiveDefinite(sigma): sigma all diagonal elements must be positive


In [31]:
head(imputed_cell_differential_plasma_post_df)

Original_Subject_No,Subject_No,Subject_ID,Disease_Status,Sample_Type,Time_Point,Inflammatory_Responder,Lung_Responder,Category,Variable,Value
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Sample_Weight,817.00
4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,TCC,395000.00
4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,TCC_Weight,483.48
4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Neutrophils_Percent,25.00
4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Neutrophils,98750.00
4,1,AS_1,AS,P,Post,NR,NR,Cell_Differential,Neutrophils_Weight,121.00
